# Vector Search with IRIS SQL
This tutorial covers how to use IRIS as a vector database. 

For this tutorial, we will use a dataset of 2.2k online reviews of scotch (
dataset from https://www.kaggle.com/datasets/koki25ando/22000-scotch-whisky-reviews) . With our latest vector database functionality, we can leverage the latest embedding models to run semantic search on the online reviews of scotch whiskeys. In addition, we'll be able to apply filters on columns with structured data. For example, we will be able to search for whiskeys that are priced under $100, and are 'earthy, smooth, and easy to drink'. Let's find our perfect whiskey!

First step is to do some imports and establish a connection to InterSystems IRIS.

In [1]:
import os, pandas as pd
from sentence_transformers import SentenceTransformer
from sqlalchemy import create_engine, text

username = 'demo'
password = 'demo'
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '1972' 
namespace = 'USER'

CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"

engine = create_engine(CONNECTION_STRING)

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Exploring the dataset

Let's take a look at the data in our CSV file with whiskey reviews.

In [2]:
# Load the CSV file
df = pd.read_csv('../data/scotch_review.csv')
df.head()

,Unnamed: 0,name,category,review.point,price,currency,description
0,1,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,$,"Magnificently powerful and intense. Caramels, ..."
1,2,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,$,What impresses me most is how this whisky evol...
2,3,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,$,There have been some legendary Bowmores from t...
3,4,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,$,With a name inspired by a 1926 Buster Keaton m...
4,5,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,$,"Captivating, enticing, and wonderfully charmin..."


Now we'll reorganize the data a little bit with panda functions to make it more practical to store in a table.

In [3]:
# Clean data
# Remove the specified columns
df.drop(['currency'], axis=1, inplace=True)

# Drop the first column
df.drop(columns=df.columns[0], inplace=True)

# Remove rows without a price
df.dropna(subset=['price'], inplace=True)

# Ensure values in 'price' are numbers
df = df[pd.to_numeric(df['price'], errors='coerce').notna()]

# Replace NaN values in other columns with an empty string
df.fillna('', inplace=True)

df.head()

,name,category,review.point,price,description
0,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,"Magnificently powerful and intense. Caramels, ..."
1,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,What impresses me most is how this whisky evol...
2,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,There have been some legendary Bowmores from t...
3,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,With a name inspired by a 1926 Buster Keaton m...
4,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,"Captivating, enticing, and wonderfully charmin..."


## Creating the table in IRIS SQL

Now, InterSystems IRIS supports vectors as a datatype in tables! Here, we create a table with a few different columns. The last column, `description_vector` of type `VECTOR(FLOAT, 384)`, will be used to store vectors that are generated by passing the `description` of a review through an embedding model. The `FLOAT` option here is new in 2024.3, and 384 is the number of dimensions the chosen embedding model uses.

In [4]:
with engine.connect() as conn:
    with conn.begin():# Load 
        sql = f"""
                CREATE TABLE IF NOT EXISTS scotch_reviews (
                    name VARCHAR(255),
                    category VARCHAR(255),
                    review_point INT,
                    price DOUBLE,
                    description VARCHAR(2000),
                    description_vector VECTOR(FLOAT, 384)
                )
                """
        result = conn.execute(text(sql))

## Creating the embeddings

Next, we'll create the embeddings for the `description` column. In IRIS 2024.3, you can leave this work to IRIS by using the new [`EMBEDDING` datatype](https://docs.intersystems.com/iris20243/csp/docbook/DocBook.UI.Page.cls?KEY=GSQL_vecsearch#GSQL_vecsearch_insembed), but for now we'll go with classic Pythonic ways of creating them, based on a common Sentence Transformer model.

In [5]:
# Load a pre-trained sentence transformer model. This model's output vectors are of size 384
model = SentenceTransformer('all-MiniLM-L6-v2') 

In [6]:
# Generate embeddings for all descriptions at once.
# Batch processing before inserting into the table makes it faster, but this step may still take a minute
embeddings = model.encode(df['description'].tolist(), normalize_embeddings=True)

# Add the embeddings to the DataFrame
df['description_vector'] = embeddings.tolist()

df.head()

,name,category,review.point,price,description,description_vector
0,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,"Magnificently powerful and intense. Caramels, ...","[-0.010494351387023926, 0.014728977344930172, ..."
1,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,What impresses me most is how this whisky evol...,"[0.02318120375275612, -0.05123031139373779, 0...."
2,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,There have been some legendary Bowmores from t...,"[0.043333109468221664, -0.017066607251763344, ..."
3,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,With a name inspired by a 1926 Buster Keaton m...,"[-0.07594006508588791, -0.036762334406375885, ..."
4,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,"Captivating, enticing, and wonderfully charmin...","[-0.012818790972232819, -0.09769780933856964, ..."


Now we'll load the data into our table. Note the `str()` call as we're passing the vector as a comma-separated list of values in string format, because there is no specific vector datatype in the DB-API driver standard.

In [7]:
with engine.connect() as conn:
    with conn.begin():
        for index, row in df.iterrows():
            sql = text("""
                INSERT INTO scotch_reviews 
                (name, category, review_point, price, description, description_vector) 
                VALUES (:name, :category, :review_point, :price, :description, TO_VECTOR(:description_vector))
            """)
            conn.execute(sql, {
                'name': row['name'], 
                'category': row['category'], 
                'review_point': row['review.point'], 
                'price': row['price'], 
                'description': row['description'], 
                'description_vector': str(row['description_vector'])
            })


## Running a few queries

Let's look for a scotch that costs less than $100, and has an earthy and creamy taste.

In [8]:
description_search = "earthy and creamy taste"
search_vector = model.encode(description_search, normalize_embeddings=True).tolist() # Convert search phrase into a vector

In [9]:
with engine.connect() as conn:
    with conn.begin():
        sql = text("""
            SELECT TOP 3 * FROM scotch_reviews 
            WHERE price < 100 
            ORDER BY VECTOR_DOT_PRODUCT(description_vector, TO_VECTOR(:search_vector)) DESC
        """)

        results = conn.execute(sql, {'search_vector': str(search_vector)}).fetchall()

print(results)

[('Signatory (distilled at Bowmore), 16 year old, 1988 vintage, cask #42508, 46%', 'Single Malt Scotch', 87, 60.0, 'Medium-bodied and nicely textured. Good balance of flavors -- and well-integrated, too -- with lovely sweet notes (cereal grain, cookie dough, carame ... (48 characters truncated) ...  fishnets, and brine that is complementary, but not aggressive, with a suggestion of lavender and tangerine. Balanced finish. (332 bottles produced.)', '-.048620406538248062133,-.082065738737583160401,.039660729467868804931,-.018970886245369911193,-.017485288903117179871,.042453136295080184936,.046325 ... (8851 characters truncated) ... 975586,-.0038619360420852899551,-.022344691678881645202,.052769239991903305053,-.061306394636631011962,.048756893724203109741,-.063436642289161682128'), ('Signatory (distilled at Bowmore), 16 year old, 1988 vintage, cask #42508, 46%', 'Single Malt Scotch', 87, 60.0, 'Medium-bodied and nicely textured. Good balance of flavors -- and well-integrated, too -- wit

Let's print that result a little more nicely!

In [10]:
results_df = pd.DataFrame(results, columns=df.columns).iloc[:, :-1] # Remove vector
pd.set_option('display.max_colwidth', None)  # Easier to read description
results_df.head()

,name,category,review.point,price,description
0,"Signatory (distilled at Bowmore), 16 year old, 1988 vintage, cask #42508, 46%",Single Malt Scotch,87,60.0,"Medium-bodied and nicely textured. Good balance of flavors -- and well-integrated, too -- with lovely sweet notes (cereal grain, cookie dough, caramel, and vanilla cream), young heathery peat, tar, fishnets, and brine that is complementary, but not aggressive, with a suggestion of lavender and tangerine. Balanced finish. (332 bottles produced.)"
1,"Signatory (distilled at Bowmore), 16 year old, 1988 vintage, cask #42508, 46%",Single Malt Scotch,87,60.0,"Medium-bodied and nicely textured. Good balance of flavors -- and well-integrated, too -- with lovely sweet notes (cereal grain, cookie dough, caramel, and vanilla cream), young heathery peat, tar, fishnets, and brine that is complementary, but not aggressive, with a suggestion of lavender and tangerine. Balanced finish. (332 bottles produced.)"
2,"Shieldaig 12 year old, 40%",Blended Scotch Whisky,85,31.0,"This is a sharp dresser, with a firm, solid mouthfeel and an altogether finer and more focused taste than Shieldaig Classic (see\r\nbelow). It’s not coastal or earthy particularly, either. Instead the flavors are softer and built around mocha, smooth creamy toffee, and some soft fruit, including a touch of overripe banana and melon notes. The savoriness this time comes from a touch of pepper rather than salt."


## Indexing vector data

The latest version of IRIS 2025.1 released through the [Early Access Program](https://www.intersystems.com/early-access-program) includes not only bug fixes and performance enhancements, but a new disk-based Approximate Nearest Neighbors index that speeds up vector search for large collections of vectors (typically over 100K). See [the docs](https://docs.intersystems.com/iris20251/csp/docbook/DocBook.UI.Page.cls?KEY=GSQL_vecsearch#GSQL_vecsearch_index) for more information on how to define and use the index.

```SQL
CREATE INDEX HNSWIndex ON TABLE scotch_reviews (description_vector) AS HNSW(M=80, Distance='DotProduct');
```

The index will automatically get used if you issue a query that uses a `TOP` clause and an `ORDER BY` to sort by the distance function for which the index was created. You can verify its use in the query plan, by using the `EXPLAIN` command or checking the plan through the System Management Portal UI.

```SQL
SELECT TOP 10 * FROM scotch_reviews ORDER BY VECTOR_DOT_PRODUCT(description_vector, TO_VECTOR(:search_vector)) DESC;
```

Since this notebook is working with a dataset much smaller than 100K rows, there won't be a measurable performance benefit, and this is provided as an example you can adapt.

NOTE: Please note this feature is currently targeted at 2025.1. Please join the [Early Access Program](https://live.evaluation.iscinternal.com/download/adminearlyaccess.csp?earlyAccessProgram=Vector_Search) if you'd like to work with a preview kit.